# Responses to the problem set
The following notebook documents my responses to the questions listed in problems.pdf

## Problem 1

MYD88 L265P is a common mutation in B cell lymphomas. A liquid biopsy found 5 of 1,000 molecules at the
locus have this mutant allele.

a) Mutant allele fraction (MAF) for this problem is 5/1000. Assuming the generation of mutants follows a binomial distribution, we could use the [wald interval](https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval
) or normal approximation to the binomial distribution to get the confidence interval for the MAF.

b) The standard deviation for the MAF is given below. If we acquired the same number of molecules, then we multiply N number of molecules by the MAF standard deviation to get its units in molecules.

The formula for the wald interval and answers to a and b are below:

In [12]:
p = MAF = 5/1000
n = 1000
std = (p*(1-p)/n)**0.5
z = 1.96 # z-score for 95% confidence interval around the mean p
p_low = p - z*std
p_high = p + z*std
print('p = ', p)
print('p_std = ',std)
print('confidence interval = [',p_low,',',p_high,']')
print('n molecules = ',n)
print('std in molecules', std*n)

p =  0.005
p_std =  0.0022304708023195463
confidence interval = [ 0.0006282772274536896 , 0.00937172277254631 ]
n molecules =  1000
std in molecules 2.2304708023195463


## Problem 2

### a) Without providing a fix, identify at least one bug in what ChatGPT wrote.

`count_fragments.py` only examines the mutations contained in the first read. It completely ignores mutations the mate pair may also have. This would make sense if targeted sequencing was done and only read1 can align to the gene of interest. Here read2 may not contain any interesting information.

If shotgun sequencing was performed instead of targeted sequencing, there is a possibility both reads can align to the gene of interest and contain relevant mutation count information.

The code also does not address what happens if read1 and read2 overlap in the same region but disagree in what they say about the same bp position. This should be unlikely if the quality threshold is set high enough to filter out low quality reads.

When we are counting mutations in a sorted bam file, we are usually interested in knowing how many mutations are at each chromosome location or bin. This code assumes all reads are reporting mutations from roughly the same locus or chromosome location.

There is no option to filter for reads at a particular chromosome location. There is no warning about the assumptions this code is making. Examining the bam file with `samtools view example.bam` showed the reads were all aligning to roughly to the same chromosome location, which means it works as intended for this piece of data. However, you can never predict how a user could misuse this code because they forget the fine details about what sort of input data this code is suppose to take.

### b) Modify this function to be a script that can be called from a shell.

see `count_fragments_zc.py` for my modifications.

### c) Without implementing anything, discuss your preferred methods to ensure the accuracy and repeatability of this program as others use and modify it. Use 100 words or fewer.

To address the concerns I outlined in A above, I would write additional docstrings to detail what exactly is the kind of input data this code is ingesting and what assumptions it is making about the input data. Next, a set of unittests are needed to check for basics like proper type handling, proper file handling, and other annoying edge cases that could come up in the development. Ideally, this code should conform to the input and output rule conventions like in snakemake, where we have a file input and file output that can be passed to downstream operations.

All these pieces noted above can help us robustly integrate this simple code into more complex pipelines.

## Problem 3